# Main Aggregation Script - calls lab.py and crystals.py

In [ ]:
#Reloads the lab.py and crystals.py modules to update any changes (after saving)
#If a new method or object is created, autoreload doesn't work and the 
#kernel needs to be closed and halted after saving and making a 'checkpoint'
#in this notebook

%load_ext autoreload
%autoreload 2

In [ ]:
import ipas 
import numpy as np
import dask
from dask_jobqueue import SLURMCluster
from dask.distributed import Client, progress
from dask import delayed
from dask import dataframe as dd
import functools
import sys
import ast
from struct import *
import pickle
import glob
import random
import pandas as pd
import time
from dask.distributed import as_completed
from joblib import Parallel, delayed, parallel_backend
import matplotlib.pyplot as plt

In [ ]:
cluster = SLURMCluster(
    queue='kratos',
    walltime='04-23:00:00',
    cores=1,
    memory='10000MiB', #1 GiB = 1,024 MiB
    processes=1)

#cluster.adapt(minimum=3, maximum=20)
cluster.scale(10)

In [ ]:
client = Client(cluster)

In [ ]:
client

# Initialize databases for queries

In [ ]:
files = [f for f in glob.glob("../instance_files/createdb_iceagg_rand*")]

In [ ]:
def shape(a,b,c):
    if (b-c) <= (a-b):
        return 'prolate'
    else:
        return 'oblate'

In [ ]:
%%time 
data = []
for file in files:
    print(file)
    #dictionary = pickle.load(f)
    data.append(pd.read_pickle(file, None))
datapd = [pd.DataFrame(i) for i in data]
df = pd.concat(datapd, axis=0, ignore_index=True)

In [ ]:
#speed up shape function 
vfunc = np.vectorize(shape)
df['shape'] = vfunc(df['a'], df['b'], df['c'])
df['agg_phi'] = df.c/df.a
df = df.reset_index()
df.loc[df_rand['shape'] == 'oblate', 'agg_r'] = np.power((np.power(df['a'], 2) * df['c']), (1./3.))
df.loc[df_rand['shape'] == 'prolate', 'agg_r'] = np.power((np.power(df['c'], 2) * df['a']), (1./3.))
df = df[df.agg_r < 5000]

# Main 

In [ ]:
ch_dist='gamma'         #anything other than gamma uses the characteristic from the best distribution pdf (lowest SSE)
rand_orient = True      #randomly orient the seed crystal and new crystal: uses first random orientation
save_plots = False 
agg_phi_bins = 20
agg_r_bins = 20
nclusters = 300

In [ ]:
def main():
    
    output = np.empty((agg_phi_bins,agg_r_bins),dtype=object)
    hold_clusters1  = np.empty((agg_phi_bins,agg_r_bins,nclusters), dtype=object)
    hold_clusters2  = np.empty((agg_phi_bins,agg_r_bins,nclusters), dtype=object)

    res, phi_bins = pd.qcut(df.agg_phi, agg_phi_bins, retbins=True)

    for i in range(1, agg_phi_bins):
        #print('agg phi range: ', phi_bins[i], phi_bins[i+1])
        #return a df that only queries within an aspect ratio bin
        df_phi = df[(df.agg_phi > phi_bins[i]) & (df.agg_phi < phi_bins[i+1])]  
        #to ensure at least 2 crystals within agg since ncrystals=1 not in db
        #now break that aspect ratio bin into 20 equal r bins
        
        res, r_bins = pd.qcut(df_phi.agg_r, agg_r_bins, retbins=True)
        for r in range(agg_r_bins):   #agg r
               
            #print('r = ', r_bins[r], r_bins[r+1])
            df_r = df_phi[(df_phi.agg_r > r_bins[r]) & (df_phi.agg_r < r_bins[r+1])]
            #plt.hist(df_r.mono_phi)
            #plt.xscale('log')
            #plt.show()

            samples1 = df_r.sample(nclusters)
            samples2 = df_r.sample(nclusters)
            
            for n, agg in enumerate(samples1.itertuples()):
                hold_clusters1[i,r,n] = ipas.Cluster_Calculations(agg)
            for n, agg in enumerate(samples2.itertuples()):
                hold_clusters2[i,r,n] = ipas.Cluster_Calculations(agg)
            ipas.collect_clusters(hold_clusters1[i,r,:], hold_clusters2[i,r,:], rand_orient=rand_orient)
            #output[i,r] = dask.delayed(ipas.collect_clusters)(hold_clusters1[i,r,:],
            #                                                     hold_clusters2[i,r,:], rand_orient=rand_orient)

    return output, hold_clusters1, hold_clusters2
    

In [ ]:
def compute():
    agg_as = np.empty((agg_phi_bins, agg_r_bins, nclusters))
    agg_bs = np.empty((agg_phi_bins, agg_r_bins, nclusters))
    rzs = np.empty((agg_phi_bins, agg_r_bins, nclusters))
    phi2Ds = np.empty((agg_phi_bins, agg_r_bins, nclusters))
    cplxs = np.empty((agg_phi_bins, agg_r_bins, nclusters))
    dds = np.empty((agg_phi_bins, agg_r_bins, nclusters))

    gather = client.compute([*output.tolist()]) 
    gather = client.gather(gather)
    gather = np.array(gather)
    print(np.shape(gather))
    agg_as = gather[:,:,0,:]
    agg_bs = gather[:,:,1,:]
    agg_cs = gather[:,:,2,:]
    phi2Ds = gather[:,:,3,:]
    cplxs = gather[:,:,4,:] 
    dds = gather[:,:,5,:]

    print('DONE!')
    return agg_as, agg_bs, agg_cs, phi2Ds, cplxs, dds

In [ ]:
if __name__ == '__main__':
    
    output, hold_clusters1, hold_clusters2 = main()
    agg_as, agg_bs, agg_cs, phi2Ds, cplxs, dds= compute()
    results = {'agg_as': agg_as, 'agg_bs':agg_bs, 'agg_cs':agg_cs, 'phi2Ds':phi2Ds, \
               'cplxs':cplxs, 'dds':dds}


In [ ]:
filename = '../instance_files/pulled_clusters_aggagg_rand'
filehandler = open(filename, 'wb')
to_file = np.append(hold_clusters1, hold_clusters2).reshape(20,20,nclusters*2)
pickle.dump(to_file, filehandler)
filehandler.close()
print('finished!')

filename = '../instance_files/instance_db_aggagg_rand'
filehandler = open(filename, 'wb')
pickle.dump(results, filehandler)
filehandler.close()
print('finished!')

# LOAD DATA

In [ ]:
f = open('../instance_files/instance_db_aggagg_rand', 'rb')
results = pickle.load(f)
agg_as, agg_bs, agg_cs, phi2Ds, cplxs, dds= \
                results['agg_as'], results['agg_bs'], results['agg_cs'], results['phi2Ds'], results['cplxs'], results['dds']
f.close()

f = open('../instance_files/pulled_clusters_aggagg_rand', 'rb')
pulled_clus = pickle.load(f)
f.close()

In [ ]:
f = open('../instance_files/pulled_clusters_iceagg_flat', 'rb')
pulled_clus = pickle.load(f)
f.close()

In [ ]:
pulled_clus.shape

In [ ]:
pulled_clus[0,0,0].__dict__

In [ ]:
count=0
for phi in range(pulled_clus.shape[0]):
    for r in range(pulled_clus.shape[1]):
        for n in range(pulled_clus.shape[2]):
            if pulled_clus[phi,r,n].monophi>70.:
                count+=1
print(count/(20*20*600)*100)

In [ ]:
count=0
for phi in range(pulled_clus.shape[0]):
    for r in range(pulled_clus.shape[1]):
        for n in range(pulled_clus.shape[2]):
            if pulled_clus[phi,r,n].monophi<.1:
                count+=1
print(count/(20*20*600)*100)